In [1]:
import numpy as np
import pandas as pd
import json
import matplotlib.pyplot as plt
import datetime
np.random_state=42
from scipy import stats
from sklearn.linear_model import LinearRegression,Lasso
import statsmodels.api as sm
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.formula.api import ols


In [2]:
finalPredictionsValid = pd.read_csv("../data/expTwoFinalPredictionsValid1125.csv")
infosValid = pd.read_csv("../data/expTwoInfosValid1125.csv")
workerIdList = list(set(finalPredictionsValid["workerId"]))
all_instances = pd.read_csv("../rawData/datingData/all_instances.csv")

In [3]:
res = pd.DataFrame()
for elem in workerIdList:

    temp = pd.DataFrame()
    workerAnswerPhaseOne = finalPredictionsValid.query("workerId=='"+str(elem)+"'").query("taskId<22")
    workerAnswerPhaseTwo = finalPredictionsValid.query("workerId=='"+str(elem)+"'").query("taskId>=22")
    allDisSwitch = np.sum(finalPredictionsValid.query("workerId=='"+str(elem)+"'")["selfPrediction"]==finalPredictionsValid.query("workerId=='"+str(elem)+"'")["finalPrediction"])
    #     tempA
    workerInfo = infosValid.query("workerId=='"+str(elem)+"'")
    
    phaseOneAgreementFraction = np.mean(workerAnswerPhaseOne["finalAgreement"])
    phaseOneSelfAccuracy = np.mean(workerAnswerPhaseOne["selfCorrect"])
    temp["trustSurvey"] = infosValid.query("workerId=='"+str(elem)+"'")["surveyTrust"].values
    temp["actualAgreement"] = [np.mean(workerAnswerPhaseOne["agreement"])]
    
    finalAgree1 = workerAnswerPhaseTwo.query("selfPrediction==1").query("agreement==0").query("finalPrediction==0")
    finalAgree2 = workerAnswerPhaseTwo.query("selfPrediction==0").query("agreement==0").query("finalPrediction==1")
    finalAgree3 = workerAnswerPhaseTwo.query("selfPrediction==1").query("agreement==1").query("finalPrediction==1")
    finalAgree4 = workerAnswerPhaseTwo.query("selfPrediction==0").query("agreement==1").query("finalPrediction==0")
    agreementFraction = (len(finalAgree1)+len(finalAgree2)+len(finalAgree3)+len(finalAgree4))/10
    agreementFraction = np.mean(workerAnswerPhaseTwo["finalAgreement"])
    
    disagreement = workerAnswerPhaseTwo.query("agreement==0")
    switchFromOne = workerAnswerPhaseTwo.query("agreement==0").query("selfPrediction==1").query("finalPrediction==0")
    switchFromZero = workerAnswerPhaseTwo.query("agreement==0").query("selfPrediction==0").query("finalPrediction==1")
    switchWhenAgreeFromZero = workerAnswerPhaseTwo.query("agreement==1").query("selfPrediction==0").query("finalPrediction==1")
    switchWhenAgreeFromOne = workerAnswerPhaseTwo.query("agreement==1").query("selfPrediction==1").query("finalPrediction==0")
    
    disAgreeSwitch = len(finalAgree1)+len(finalAgree2)
    agreeDisSwitch = len(finalAgree3)+len(finalAgree4)

    agreeSwitch = len(switchWhenAgreeFromZero)+len(switchWhenAgreeFromOne)
    
    disAgreeDisSwitch = len(
        workerAnswerPhaseTwo.query("selfPrediction==1").query("agreement==0").query("finalPrediction==1"))+len(
        workerAnswerPhaseTwo.query("selfPrediction==0").query("agreement==0").query("finalPrediction==0"))
    
    if len(disagreement)!=0:
        switchFraction = (len(switchFromOne)+len(switchFromZero))/len(disagreement)
    else:
        switchFraction = -1
    switchWhenAgree = len(switchWhenAgreeFromZero)+len(switchWhenAgreeFromOne)
    finalAccuracy = np.mean(workerAnswerPhaseTwo["finalCorrect"])
    temp["agreeDisSwitch"] = [agreeDisSwitch]
    temp["agreeSwitch"] = [agreeSwitch]
    temp["disAgreeSwitch"] = [disAgreeSwitch]
    temp["disAgreeDisSwitch"] = [disAgreeDisSwitch]
    temp["allDisSwitch"] = [allDisSwitch]
    temp["finalAccuracy"] = [finalAccuracy]      
    temp["workerId"] = [elem]  
    temp["agreementFraction"] = [agreementFraction]
    temp["switchFraction"] = [switchFraction]

    idpAgreement = np.mean(workerAnswerPhaseTwo["idpAgreement"])
    acc = np.mean(workerAnswerPhaseTwo["acc"])
    treatment = np.mean(workerAnswerPhaseTwo["treatment"])
    temp["idpAgreement"] = idpAgreement
    temp["acc"] = acc
    temp["treatment"] = treatment
    temp["competence"] = [int(workerInfo["competence"])]
    temp["faith"] = [int(workerInfo["faith"])]
    temp["understandability"] = [int(workerInfo["understandability"])]
    temp["reliability"] = [int(workerInfo["reliability"])]
    res = res.append(temp)

In [4]:
supplementary_res = pd.DataFrame()
for elem in workerIdList:
    temp = pd.DataFrame()
    
    workerAnswerAll = finalPredictionsValid.query("workerId=='"+str(elem)+"'")
    workerAnswerPhaseOne = finalPredictionsValid.query("workerId=='"+str(elem)+"'").query("taskId<22")
    workerAnswerPhaseTwo = finalPredictionsValid.query("workerId=='"+str(elem)+"'").query("taskId>=22")
    workerAnswerPhaseOnePartThree = finalPredictionsValid.query("workerId=='"+str(elem)+"'").query("taskId>=11").query("taskId<=15")
    workerAnswerPhaseOnePartFour = finalPredictionsValid.query("workerId=='"+str(elem)+"'").query("taskId>15").query("taskId<=22")

    phaseOneSelfAccuracy = np.mean(workerAnswerPhaseOne["selfCorrect"])
    phaseTwoSelfAccuracy = np.mean(workerAnswerPhaseTwo["selfCorrect"])
    
    phaseOneFinalAccuracy = np.mean(workerAnswerPhaseOne["finalCorrect"])
    phaseTwoFinalAccuracy = np.mean(workerAnswerPhaseTwo["finalCorrect"])
    
    phaseOneSelfAccuracyMlcorrect = np.mean(workerAnswerPhaseOne.query("mlCorrect==True")["selfCorrect"])
    phaseOneSelfAccuracyMlincorrect = np.mean(workerAnswerPhaseOne.query("mlCorrect==False")["selfCorrect"])
    phaseTwoSelfAccuracyMlcorrect = np.mean(workerAnswerPhaseTwo.query("mlCorrect==True")["selfCorrect"])
    phaseTwoSelfAccuracyMlincorrect = np.mean(workerAnswerPhaseTwo.query("mlCorrect==False")["selfCorrect"])
    
    phaseOneFinalAccuracyMlcorrect = np.mean(workerAnswerPhaseOne.query("mlCorrect==True")["finalCorrect"])
    phaseOneFinalAccuracyMlincorrect = np.mean(workerAnswerPhaseOne.query("mlCorrect==False")["finalCorrect"])
    phaseTwoFinalAccuracyMlcorrect = np.mean(workerAnswerPhaseTwo.query("mlCorrect==True")["finalCorrect"])
    phaseTwoFinalAccuracyMlincorrect = np.mean(workerAnswerPhaseTwo.query("mlCorrect==False")["finalCorrect"])   
    
    phaseTwoAgreementFrationMlCorrect = np.mean(workerAnswerPhaseTwo.query("mlCorrect==True")["finalAgreement"])
    phaseTwoAgreementFrationMlIncorrect = np.mean(workerAnswerPhaseTwo.query("mlCorrect==False")["finalAgreement"])
    phaseTwoSwitchFractionM1Correct = np.mean((workerAnswerPhaseTwo.query("mlCorrect==True")["switch"]))
    phaseTwoSwitchFractionM1Incorrect = np.mean((workerAnswerPhaseTwo.query("mlCorrect==False")["switch"]))

    idpAgreementPhaseOnePartThree = np.mean(workerAnswerPhaseOnePartThree["agreement"])
    idpAgreementPhaseOnePartFour = np.mean(workerAnswerPhaseOnePartFour["agreement"])

    allSelfAccuracy = np.mean(workerAnswerAll["selfCorrect"])
    allFinalAccuracy = np.mean(workerAnswerAll["finalCorrect"])
    allFinalAccuracyMlcorrect = np.mean(workerAnswerAll.query("mlCorrect==True")["finalCorrect"])
    allFinalAccuracyMlincorrect = np.mean(workerAnswerAll.query("mlCorrect==False")["finalCorrect"])
    

    temp["workerId"] = [elem]
    temp["phaseTwoActualAgreement"] = np.mean(workerAnswerPhaseTwo["agreement"])
    temp["idpAgreement"] = [workerAnswerPhaseOne["idpAgreement"].values[0]] 
                                 
    temp["phaseTwoAgreementFrationMlCorrect"] = [phaseTwoAgreementFrationMlCorrect]      
    temp["phaseTwoAgreementFrationMlIncorrect"] = [phaseTwoAgreementFrationMlIncorrect]      
    temp["phaseTwoSwitchFractionM1Correct"] = [phaseTwoSwitchFractionM1Correct]      
    temp["phaseTwoSwitchFractionM1Incorrect"] = [phaseTwoSwitchFractionM1Incorrect]      
    temp["idpAgreementPhaseOnePartThree"] = [idpAgreementPhaseOnePartThree] 
    temp["idpAgreementPhaseOnePartFour"] = [idpAgreementPhaseOnePartFour] 

    temp["allSelfAccuracy"] = [allSelfAccuracy]      
    temp["allFinalAccuracy"] = [allFinalAccuracy]  
    temp["allFinalAccuracyMlcorrect"] = [allFinalAccuracyMlcorrect]  
    temp["allFinalAccuracyMlincorrect"] = [allFinalAccuracyMlincorrect]  
    temp["phaseOneSelfAccuracy"] = [phaseOneSelfAccuracy]      
    temp["phaseTwoSelfAccuracy"] = [phaseTwoSelfAccuracy]      
    temp["phaseOneFinalAccuracy"] = [phaseOneFinalAccuracy]      
    temp["phaseTwoFinalAccuracy"] = [phaseTwoFinalAccuracy]
    temp["phaseOneSelfAccuracyMlcorrect"] = [phaseOneSelfAccuracyMlcorrect]      
    temp["phaseOneSelfAccuracyMlincorrect"] = [phaseOneSelfAccuracyMlincorrect]  
    temp["phaseTwoSelfAccuracyMlcorrect"] = [phaseTwoSelfAccuracyMlcorrect]      
    temp["phaseTwoSelfAccuracyMlincorrect"] = [phaseTwoSelfAccuracyMlincorrect]
    temp["phaseOneFinalAccuracyMlcorrect"] = [phaseOneFinalAccuracyMlcorrect]      
    temp["phaseOneFinalAccuracyMlincorrect"] = [phaseOneFinalAccuracyMlincorrect]      
    temp["phaseTwoFinalAccuracyMlcorrect"] = [phaseTwoFinalAccuracyMlcorrect]      
    temp["phaseTwoFinalAccuracyMlincorrect"] = [phaseTwoFinalAccuracyMlincorrect]      
    supplementary_res = supplementary_res.append(temp)

In [5]:
treatmentLabelDict = {0:"agree50acc50",1:"agree80acc50",2:"agree50acc80",3:"agree80acc80"}
res["treatmentLabel"] = res["treatment"].apply(lambda x: treatmentLabelDict[x])

In [6]:
all_res = pd.merge(res,supplementary_res,on=["workerId","idpAgreement"])
res.to_csv("../data/exp2Res1125.csv")

res

In [7]:
group_list = [res.query("treatment==@i") for i in range(4)]
print("ANOVA on actual agreement (low acc)",
      ":",stats.f_oneway(group_list[0]["actualAgreement"],group_list[1]["actualAgreement"] ))
print("ANOVA on actual agreement (high acc)",
      ":",stats.f_oneway(group_list[2]["actualAgreement"],group_list[3]["actualAgreement"] ))

[np.mean(elem["actualAgreement"]) for elem in group_list]

ANOVA on actual agreement (low acc) : F_onewayResult(statistic=454.5516850223174, pvalue=3.549372823100622e-58)
ANOVA on actual agreement (high acc) : F_onewayResult(statistic=262.51301074532574, pvalue=5.949558833058414e-39)


[0.5225190839694657,
 0.7566115702479332,
 0.5216814159292035,
 0.7242574257425738]

In [8]:
res_anova_dp_list = ["agreementFraction","switchFraction","phaseTwoAgreementFrationMlCorrect","phaseTwoSwitchFractionM1Correct","phaseTwoAgreementFrationMlIncorrect","phaseTwoSwitchFractionM1Incorrect"]

In [9]:
def assumption_test(df,dp_list):
    for elem in dp_list:
        for group in range(4):
            print(group,"normality", stats.shapiro(df.query("treatment==@group")[elem]))
        print("homogeneity",stats.bartlett(df.query("treatment==0")[elem],df.query("treatment==1")[elem],df.query("treatment==2")[elem],df.query("treatment==3")[elem]))


In [10]:
assumption_test(all_res,res_anova_dp_list)

0 normality ShapiroResult(statistic=0.965834379196167, pvalue=0.0022112990263849497)
1 normality ShapiroResult(statistic=0.9545736908912659, pvalue=0.0004470158601179719)
2 normality ShapiroResult(statistic=0.9419673681259155, pvalue=9.659421630203724e-05)
3 normality ShapiroResult(statistic=0.9607290029525757, pvalue=0.004277494736015797)
homogeneity BartlettResult(statistic=3.411262307075858, pvalue=0.3324547759251432)
0 normality ShapiroResult(statistic=0.7604734301567078, pvalue=2.4170482238947133e-13)
1 normality ShapiroResult(statistic=0.7631611824035645, pvalue=1.0848391412618397e-12)
2 normality ShapiroResult(statistic=0.8957275152206421, pvalue=2.3717984731774777e-07)
3 normality ShapiroResult(statistic=0.8046672344207764, pvalue=3.007481763628306e-10)
homogeneity BartlettResult(statistic=3.425112281334544, pvalue=0.33060547619800473)
0 normality ShapiroResult(statistic=0.944718062877655, pvalue=4.255482053849846e-05)
1 normality ShapiroResult(statistic=0.9523558616638184, pva

### main effect

In [11]:
def anova(df,measure):
    group0 = df.query("treatment==0")[measure]
    group1 = df.query("treatment==1")[measure]
    group2 = df.query("treatment==2")[measure]
    group3 = df.query("treatment==3")[measure]
    moore_lm = ols(measure+' ~ C(idpAgreement)*C(acc)', data=df).fit()
    table = sm.stats.anova_lm(moore_lm, typ=2)
    print(table)
    m_comp = pairwise_tukeyhsd(endog=df[measure], groups=df['treatmentLabel'], alpha=0.05)
    print(m_comp)

In [12]:
anova(all_res,"agreementFraction")
anova(all_res,"switchFraction")

                           sum_sq     df          F    PR(>F)
C(idpAgreement)          0.053453    1.0   1.255777  0.263034
C(acc)                   0.749712    1.0  17.613043  0.000032
C(idpAgreement):C(acc)   0.014252    1.0   0.334830  0.563111
Residual                19.665370  462.0        NaN       NaN
      Multiple Comparison of Means - Tukey HSD, FWER=0.05       
   group1       group2    meandiff p-adj   lower   upper  reject
----------------------------------------------------------------
agree50acc50 agree50acc80   0.0911 0.0036  0.0228  0.1594   True
agree50acc50 agree80acc50  -0.0112    0.9 -0.0783  0.0558  False
agree50acc50 agree80acc80   0.0576 0.1521 -0.0129   0.128  False
agree50acc80 agree80acc50  -0.1023  0.001 -0.1719 -0.0327   True
agree50acc80 agree80acc80  -0.0335 0.6206 -0.1063  0.0394  False
agree80acc50 agree80acc80   0.0688 0.0652 -0.0029  0.1405  False
----------------------------------------------------------------
                           sum_sq     df

In [13]:
anova(all_res,"trustSurvey")

                             sum_sq     df          F    PR(>F)
C(idpAgreement)           14.451266    1.0   6.592735  0.010553
C(acc)                    31.862339    1.0  14.535748  0.000156
C(idpAgreement):C(acc)     2.541251    1.0   1.159330  0.282166
Residual                1012.703337  462.0        NaN       NaN
      Multiple Comparison of Means - Tukey HSD, FWER=0.05      
   group1       group2    meandiff p-adj   lower  upper  reject
---------------------------------------------------------------
agree50acc50 agree50acc80   0.3836 0.1828 -0.1065 0.8737  False
agree50acc50 agree80acc50   0.2165 0.6347 -0.2649 0.6978  False
agree50acc50 agree80acc80   0.8968  0.001  0.3912 1.4023   True
agree50acc80 agree80acc50  -0.1671  0.801 -0.6665 0.3323  False
agree50acc80 agree80acc80   0.5132 0.0565 -0.0096 1.0359  False
agree80acc50 agree80acc80   0.6803 0.0039  0.1658 1.1948   True
---------------------------------------------------------------
